# Large neighborhood search + CP to solve RCPSP
LNS is an iterative heuristic method consisting in freezing randomly part of the solutions and optimize the remaining part. Full solution is then rebuilt and hopefully, repeating the process lead to a good solution to the original problem.



## Prerequisites


Concerning the python kernel to use for this notebook:
- If running locally, be sure to use an environment with discrete-optimization and minizinc.
- If running on colab, the next cell does it for you.
- If running on binder, the environment should be ready.


In [ ]:
# On Colab: install the library
on_colab = "google.colab" in str(get_ipython())
if on_colab:
    import importlib
    import os
    import sys  # noqa: avoid having this import removed by pycln

    !{sys.executable} -m pip install -U pip

    # uninstall google protobuf conflicting with ray and sb3
    ! pip uninstall -y protobuf

    # install dev version for dev doc, or release version for release doc
    !{sys.executable} -m pip install git+https://github.com/airbus/discrete-optimization@master#egg=discrete-optimization

    # be sure to load the proper cffi (downgraded compared to the one initially on colab)
    import cffi

    importlib.reload(cffi)

    # install and configure minizinc
    !curl -o minizinc.AppImage -L https://github.com/MiniZinc/MiniZincIDE/releases/download/2.6.3/MiniZincIDE-2.6.3-x86_64.AppImage
    !chmod +x minizinc.AppImage
    !./minizinc.AppImage --appimage-extract
    os.environ["PATH"] = f"{os.getcwd()}/squashfs-root/usr/bin/:{os.environ['PATH']}"
    os.environ["LD_LIBRARY_PATH"] = (
        f"{os.getcwd()}/squashfs-root/usr/lib/:{os.environ['LD_LIBRARY_PATH']}"
    )

### Imports

In [ ]:
import logging

import matplotlib.pyplot as plt
import nest_asyncio

from discrete_optimization.datasets import fetch_data_from_psplib
from discrete_optimization.generic_rcpsp_tools.large_neighborhood_search_scheduling import (
    LargeNeighborhoodSearchScheduling,
)
from discrete_optimization.generic_rcpsp_tools.neighbor_builder import (
    ObjectiveSubproblem,
    ParamsConstraintBuilder,
    mix_lot,
)
from discrete_optimization.generic_tools.cp_tools import CPSolverName, ParametersCP
from discrete_optimization.generic_tools.lns_cp import LNS_CP
from discrete_optimization.generic_tools.lns_mip import TrivialInitialSolution
from discrete_optimization.generic_tools.result_storage.result_storage import (
    from_solutions_to_result_storage,
)
from discrete_optimization.rcpsp.rcpsp_parser import get_data_available, parse_file
from discrete_optimization.rcpsp.solver.cp_solvers import (
    CP_RCPSP_MZN,
    CPSolverName,
    ParametersCP,
)
from discrete_optimization.rcpsp.solver.rcpsp_cp_lns_solver import (
    ConstraintHandlerStartTimeInterval_CP,
)

# patch asyncio so that applications using async functions can run in jupyter
nest_asyncio.apply()

# set logging level
logging.basicConfig(level=logging.INFO)

### Download datasets

If not yet available, we import the datasets from [psplib](https://www.om-db.wi.tum.de/psplib/data.html).

In [ ]:
needed_datasets = ["j301_1.sm"]
download_needed = False
try:
    files_available_paths = get_data_available()
    for dataset in needed_datasets:
        if len([f for f in files_available_paths if dataset in f]) == 0:
            download_needed = True
            break
except:
    download_needed = True

if download_needed:
    fetch_data_from_psplib()

This time we'll use a more complicated instance of RCPSP to highlight the benefit of LNS. (For introduction about RCPSP problem, see [this notebook](RCPSP%20%231%20Introduction.ipynb).)

In [ ]:
# Parse some rcpsp file
filepath = [f for f in get_data_available() if "j1201_3.sm" in f][0]
rcpsp_problem = parse_file(filepath)

## Baseline solver 

In [ ]:
solver = CP_RCPSP_MZN(problem=rcpsp_problem, cp_solver_name=CPSolverName.CHUFFED)
params_cp = ParametersCP.default()
params_cp.time_limit = 20
solver.init_model(output_type=True)
results = solver.solve(parameters_cp=params_cp)

In [ ]:
best_solution_cp, fit = results.get_best_solution_fit()
print("Best makespan found by CP: ", -fit)

## LNS

In [ ]:
constraint_handler = ConstraintHandlerStartTimeInterval_CP(
    problem=rcpsp_problem,
    fraction_to_fix=0.8,
    # here i want to apply bounds constraint on all the tasks
    minus_delta=10,
    plus_delta=10,
)

some_solution = rcpsp_problem.get_dummy_solution()  # starting solution
initial_solution_provider = TrivialInitialSolution(
    solution=from_solutions_to_result_storage([some_solution], problem=rcpsp_problem)
)
parameters_cp = ParametersCP.default()
parameters_cp.time_limit_iter0 = 5
parameters_cp.time_limit = 2
lns_solver = LNS_CP(
    problem=rcpsp_problem,
    cp_solver=solver,
    initial_solution_provider=initial_solution_provider,
    constraint_handler=constraint_handler,
)
result_store = lns_solver.solve_lns(
    max_time_seconds=100, parameters_cp=parameters_cp, nb_iteration_lns=100
)

## Easier LNS for scheduling

Ease the use of LNS solver, with by default initial solution provider, constraint handler etc. By default parameters may work less good than customized ones.

In [ ]:
lns_solver = LargeNeighborhoodSearchScheduling(problem=rcpsp_problem)

In [ ]:
parameters_cp = ParametersCP.default()
parameters_cp.time_limit_iter0 = 5
parameters_cp.time_limit = 2
results = lns_solver.solve(
    nb_iteration_lns=1000,
    skip_first_iteration=False,
    stop_first_iteration_if_optimal=False,
    parameters_cp=parameters_cp,
    nb_iteration_no_improvement=200,
    max_time_seconds=100,
)

## Advanced LNS settings
More advanced user can use "port-folio" constraint handler. 

In [ ]:
# Different constraint handler methods
constraint_handler = mix_lot(
    rcpsp_model=rcpsp_problem,
    nb_cut_parts=[4, 5, 6],
    fraction_subproblems=[0.3],
    params_list=[
        ParamsConstraintBuilder(
            minus_delta_primary=60,
            plus_delta_primary=60,
            minus_delta_secondary=20,
            plus_delta_secondary=20,
            constraint_max_time_to_current_solution=True,
        )
    ],
    objective_subproblem=ObjectiveSubproblem.MAKESPAN_SUBTASKS,
)

This constraint handler is choosing randomly different ways of building subproblems : 
- cut parts (depending on some integer values) make the optim focused on some subpart of the current solution
- random subproblem (specified by `fraction_subproblems`) : a given fraction of the problem is set to be the main focus of the subproblem.
By specifying a list of "cut_part" and "fraction_subproblems", all the different methods are "mixed" and randomly chosen during the LNS iterations. This allows a more diverse LNS and can help the overall optimisation.

The method is more detailed in an upcoming scientific paper that will be linked in this tutorial.

In [ ]:
solver = CP_RCPSP_MZN(problem=rcpsp_problem, cp_solver_name=CPSolverName.CHUFFED)
solver.init_model(
    output_type=True, ignore_sec_objective=False, add_objective_makespan=False
)
some_solution = rcpsp_problem.get_dummy_solution()  # starting solution
initial_solution_provider = TrivialInitialSolution(
    solution=from_solutions_to_result_storage([some_solution], problem=rcpsp_problem)
)
lns_solver = LargeNeighborhoodSearchScheduling(
    problem=rcpsp_problem,
    cp_solver=solver,
    constraint_handler=constraint_handler,
    initial_solution_provider=initial_solution_provider,
)

In [ ]:
parameters_cp = ParametersCP.default()
parameters_cp.time_limit_iter0 = 5
parameters_cp.time_limit = 3
parameters_cp.free_search = True
results = lns_solver.solve(
    nb_iteration_lns=1000,
    skip_first_iteration=False,
    stop_first_iteration_if_optimal=False,
    parameters_cp=parameters_cp,
    nb_iteration_no_improvement=200,
    max_time_seconds=100,
)

In [ ]:
fig, ax = plt.subplots(1)
ax.plot([x[1] for x in results.list_solution_fits], marker="o")
ax.set_ylabel("- makespan")
ax.set_xlabel("# solution found")
plt.show()